In [19]:
import re
from bs4 import BeautifulSoup
import json

# Specify the path to your XML file
xml_file_path = 'C:\\Users\\walke\\Downloads\\19861231_086U0917_VIGENZA_20230817.xml'

# Read the XML file
with open(xml_file_path, 'r', encoding='utf-8') as xml_file:
    xml_content = xml_file.read()

# Parse the XML content using Beautiful Soup
soup = BeautifulSoup(xml_content, 'xml')

# Initialize variables to store nodes and links data
nodes = []
links = []
unique_dates = set()

# Find all <article> elements
article_elements = soup.find_all('article')

# Loop through all <article> elements
for article in article_elements:
    num_element = article.find('num')
    if num_element:
        current_article = num_element.get_text().strip()

        # Remove trailing dot from the id
        current_article = re.sub(r'\.$', '', current_article)

        nodes.append({"data": {"id": current_article, "group": 1}})

    ref_paragraphs = article.find_all('paragraph')
    if ref_paragraphs:
        for ref_tag in ref_paragraphs:
            num_element = ref_tag.find('num')
            if num_element:
                ref_text = num_element.get_text().strip()

                # Check for matching reference tags
                matching_ref_tags = ref_tag.find_all('ref')
                for matching_ref in matching_ref_tags:
                    matching_ref_text = matching_ref.get_text().strip()

                    # Extract the reference ID (e.g., "n.544") using a regular expression if it matches the format
                    reference_id = re.search(r'n\.\d+', matching_ref_text)
                    reference_id = reference_id.group() if reference_id else matching_ref_text  # Use matching_ref_text if the format doesn't match
                    reference_id = reference_id.replace(",", "")  # Remove any commas

                    year_match = re.search(r'\d{4}', matching_ref_text)
                    year = int(year_match.group()) if year_match else 0

                    # Remove "articolo" or "articoli" and their numbers from the reference
                    law_reference = re.sub(r'articolo?\s*\d+\s*', '', reference_id)
                    law_reference = re.sub(r'articoli?\s*\d+\s*', '', law_reference)

                    if any(keyword in law_reference for keyword in ['gennaio', 'febbraio', 'marzo', 'aprile', 'maggio', 'giugno', 'luglio', 'agosto', 'settembre', 'ottobre', 'novembre', 'dicembre']):
                        date_match = re.search(r'\b(?:\d{1,2}\s+(?:gennaio|febbraio|marzo|aprile|maggio|giugno|luglio|agosto|settembre|ottobre|novembre|dicembre)\s+\d{4})\b', matching_ref_text)
                        if date_match:
                            date_info = date_match.group(0)

                            # Check if date_info is not in the set before adding to nodes and links
                            if date_info not in unique_dates:
                                unique_dates.add(date_info)
                                nodes.append({"data": {"id": date_info, "group": year, "influence": 1}})
                            links.append({"data": {"source": current_article, "target": date_info, "thick": 1}})

# Prepare the final JSON structure
cytoscape_data = {
    "elements": {
        "nodes": nodes,
        "edges": links
    }
}

# Save nodes and links to a single JSON file
with open('C:\\xampp\\htdocs\\graphie-main\\graphie-main\\akomantoso.json', 'w') as graph_file:
    json.dump(cytoscape_data, graph_file, indent=2)

print("JSON file generated successfully.")


JSON file generated successfully.


In [21]:
import json

# Load the generated JSON file
json_file_path = 'C:\\xampp\\htdocs\\graphie-main\\graphie-main\\akomantoso.json'
with open(json_file_path, 'r') as json_file:
    cytoscape_data = json.load(json_file)

# Retrieve nodes and links data
nodes = cytoscape_data["elements"]["nodes"]
links = cytoscape_data["elements"]["edges"]

# Initialize a dictionary to store the count of incoming references for each node
incoming_references_count = {node["data"]["id"]: 0 for node in nodes}

# Count incoming references
for link in links:
    target_node_id = link["data"]["target"]
    incoming_references_count[target_node_id] += 1

# Update the nodes with the incoming_references metadata
for node in nodes:
    node_id = node["data"]["id"]
    node["data"]["incoming"] = incoming_references_count[node_id]

# Save the updated nodes and links to the same JSON file
with open(json_file_path, 'w') as updated_json_file:
    json.dump(cytoscape_data, updated_json_file, indent=2)

print("Updated JSON file with incoming metadata.")


Updated JSON file with incoming metadata.
